script to add a column to the reports determining gene type, and filter for genes qith seqiD > 99%

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import plotnine
import pyodbc as pyo
import datetime

from plotnine import *

In [2]:
#define fuction to determine gene type (BLEE, CPO, Other)
def determine_gene_type(gene):
    if gene in list_BLEE:
        return "BLEE"
    elif gene in list_CPO:
        return "CPO"
    else:
        return "Other"
    
os.chdir("/data/bioinfo_doc/research/20240409_TFM-CMARTINEZ_IC-SM_T/20240711_RESULTS/reports_modified")
date = datetime.datetime.now().strftime("%Y%m%d")


#Import reference lists.
data_BLEE = pd.read_csv("20240805eq_BLEE.tsv", sep = "\t")
data_CPO = pd.read_csv("20240805eq_CPO.tsv", sep = "\t")

list_BLEE = data_BLEE['blee_modified'].to_list()
list_CPO = data_CPO['cpo_modified'].to_list()

#import output data
ariba_CARD =  pd.read_csv("20240805ariba_CARD_modified.tsv", sep = "\t")
ariba_NCBI = pd.read_csv("20240805ariba_NCBI_modified.tsv", sep = "\t")
ariba_RESFINDER = pd.read_csv("20240805ariba_RESFINDER_modified.tsv", sep = "\t")
abricate_CARD = pd.read_csv("20240805abricate_CARD_modified.tsv", sep = "\t")
abricate_NCBI = pd.read_csv("20240805abricate_NCBI_modified.tsv", sep = "\t")
abricate_RESFINDER = pd.read_csv("20240805abricate_RESFINDER_modified.tsv", sep = "\t")
amrfinderplus_NCBI = pd.read_csv("20240805amrfinderplus_NCBI_modified.tsv", sep = "\t")
rgi_CARD = pd.read_csv("20240805rgi_CARD_modified.tsv", sep = "\t")

os.chdir("../")

#define a dictionary to ease iteration through reports.
reports_ariba = {
    "ariba_CARD": ariba_CARD,
    "ariba_NCBI": ariba_NCBI,
    "ariba_RESFINDER": ariba_RESFINDER
}

all_reports = {
    "ariba_CARD": ariba_CARD,
    "ariba_NCBI": ariba_NCBI,
    "ariba_RESFINDER": ariba_RESFINDER,
    "abricate_CARD": abricate_CARD,
    "abricate_NCBI": abricate_NCBI,
    "abricate_RESFINDER": abricate_RESFINDER,
    "amrfinderplus_NCBI": amrfinderplus_NCBI,
    "rgi_CARD": rgi_CARD
}

Calculating sequence id% (having dropped duplicates)

In [4]:
#Filter the found genes by %sequence ID in ALL METHODS. Store values in a list to use later
df_ids = pd.DataFrame(columns = ["unfiltered_seq_id", "filtered_seq_id"])
ids_unfiltered = []
ids_filtered99 = []
ids_filtered98 = [] #we will also check the fraction of genes that pass the 98% seqID filter, but will use only the 99% filter for the final report.

#we will be creating a dictionary to store all reports, but only containing the gene matches with a seq id >= 99%. 
all_reports_filtered99 = {}

for key, value in all_reports.items():
    ids_unfiltered.append(len(value))
    if value["sequence_identity"] is not None:
        value = value[value['sequence_identity']>=98]
        ids_filtered98.append(len(value))        
        value = value[value['sequence_identity']>=99]
        ids_filtered99.append(len(value))
        all_reports_filtered99[key] = value
    else:
        print("There are genes for which the sequence identity is not available.")
        break


#calculate fraction of initial matches that pass the seqID > 99% filter.
fraction_passed99 = []
fraction_passed98 = []
for i in range(0,len(all_reports)):
   fraction_passed99.append(ids_filtered99[i]/ids_unfiltered[i]*100)
   fraction_passed98.append(ids_filtered98[i]/ids_unfiltered[i]*100)
   
#create a dataframe to store the values.
ids_passed = pd.DataFrame(
    {"unfiltered_seq_id%": ids_unfiltered, 
     "seq_id > 99%": ids_filtered99,
     "seq_id > 98%": ids_filtered98,
     "fraction of initial matches that pass filter >= 99%": fraction_passed99,
     "fraction of initial matches that pass filter >= 98%": fraction_passed98},
    index = all_reports.keys()).round(2)

#save the data (number of genes that pass the seq ID filters) to a file.
#ids_passed.to_csv(os.path.join(date + "_seqID_filter.tsv"), sep = "\t")
ids_passed

,unfiltered_seq_id%,seq_id > 99%,seq_id > 98%,fraction of initial matches that pass filter >= 99%,fraction of initial matches that pass filter >= 98%
ariba_CARD,3548,2252,2568,63.47,72.38
ariba_NCBI,605,503,558,83.14,92.23
ariba_RESFINDER,1016,841,964,82.78,94.88
abricate_CARD,2581,1116,1354,43.24,52.46
abricate_NCBI,916,879,913,95.96,99.67
abricate_RESFINDER,913,781,906,85.54,99.23
amrfinderplus_NCBI,2817,2263,2446,80.33,86.83
rgi_CARD,126124,712,815,0.56,0.65


2. Map the genes found with the reference lists of carbapenemases and CPOs.


In [5]:
df_GOIs = pd.DataFrame(columns = ["found genes (seqID >= 99%)", "of which BLEEs/CPOs"])

#filter to see if the found gene is a match to the reference lists of BLEEs and carbapenemases.
for key, value in all_reports_filtered99.items():
        value = value[value['gene_symbol_modified'].isin(list_BLEE) | value['gene_symbol_modified'].isin(list_CPO)]
        df_GOIs.loc[key] = len(all_reports_filtered99[key]), len(value)

df_GOIs.index = all_reports_filtered99.keys() 
df_GOIs["percentage of BLEEs/CPOs"] = (df_GOIs["of which BLEEs/CPOs"]/df_GOIs["found genes (seqID >= 99%)"]*100).round(2)
#save the list of genes (already done)
#df_GOIs.to_csv(os.path.join(date +"NumberGOIs.tsv"), sep = "\t")
df_GOIs

,found genes (seqID >= 99%),of which BLEEs/CPOs,percentage of BLEEs/CPOs
ariba_CARD,2252,264,11.72
ariba_NCBI,503,241,47.91
ariba_RESFINDER,841,220,26.16
abricate_CARD,1116,256,22.94
abricate_NCBI,879,256,29.12
abricate_RESFINDER,781,217,27.78
amrfinderplus_NCBI,2263,251,11.09
rgi_CARD,712,267,37.50


3. Filter the AMR genes found that are a match with the GOIs, and create a tag (BLEE/CPO) in the report's dataframe to those genes.

In [6]:
#filter to see if the found gene is a BLEE or CPO.
for key, value in all_reports_filtered99.items():
   all_reports_filtered99[key]['gene_type'] = all_reports_filtered99[key]['gene_symbol_modified'].apply(determine_gene_type)
for key, value in all_reports.items():
   all_reports[key]['gene_type'] = all_reports[key]['gene_symbol_modified'].apply(determine_gene_type)

save the reports filtered for seq ID >= 99% and with the column BLEE/CPO added

In [ ]:
#save the reports with the [seqID >= 99%] applied. These will be the final AMR gene finding results.
path_filtered_reports = "/data/bioinfo_doc/research/20240409_TFM-CMARTINEZ_IC-SM_T/20240711_RESULTS/reports_filtered99"
if not os.path.exists(path_filtered_reports):
    os.mkdir(path_filtered_reports)

print("Saving files:")
for key, value in all_reports_filtered99.items():
    file = all_reports_filtered99[key]
    file.to_csv(os.path.join(path_filtered_reports + "/" + date + key + ".tsv"), index = False, sep = "\t")
    print(key + " saved.")

for key, value in all_reports.items():
    file = all_reports[key]
    file.to_csv(os.path.join("reports_modified/" + date + key + ".tsv"), index = False, sep = "\t")
    print(key + " saved.")

# 2. sacar un df con todas las blees/cpos detectadas al menos con una de las tools.          
deteccion = presencia conun seqid > 99%

In [3]:
import pandas as pd
import os
import datetime

#define fuction to determine gene type (BLEE, CPO, Other)
def determine_gene_type(gene):
    if gene in list_BLEE:
        return "BLEE"
    elif gene in list_CPO:
        return "CPO"
    else:
        return "Other"
    
os.chdir("/data/bioinfo_doc/research/20240409_TFM-CMARTINEZ_IC-SM_T/20240711_RESULTS/reports_modified")
date = datetime.datetime.now().strftime("%Y%m%d")


#Import reference lists.
data_BLEE = pd.read_csv("20240805eq_BLEE.tsv", sep = "\t")
data_CPO = pd.read_csv("20240805eq_CPO.tsv", sep = "\t")

list_BLEE = data_BLEE['blee_modified'].to_list()
list_CPO = data_CPO['cpo_modified'].to_list()

#import output data
ariba_CARD =  pd.read_csv("20240805ariba_CARD_modified.tsv", sep = "\t")
ariba_NCBI = pd.read_csv("20240805ariba_NCBI_modified.tsv", sep = "\t")
ariba_RESFINDER = pd.read_csv("20240805ariba_RESFINDER_modified.tsv", sep = "\t")
abricate_CARD = pd.read_csv("20240805abricate_CARD_modified.tsv", sep = "\t")
abricate_NCBI = pd.read_csv("20240805abricate_NCBI_modified.tsv", sep = "\t")
abricate_RESFINDER = pd.read_csv("20240805abricate_RESFINDER_modified.tsv", sep = "\t")
amrfinderplus_NCBI = pd.read_csv("20240805amrfinderplus_NCBI_modified.tsv", sep = "\t")
rgi_CARD = pd.read_csv("20240805rgi_CARD_modified.tsv", sep = "\t")

os.chdir("../")

#define a dictionary to ease iteration through reports.
all_reports = {
    "ariba_CARD": ariba_CARD,
    "ariba_NCBI": ariba_NCBI,
    "ariba_RESFINDER": ariba_RESFINDER,
    "abricate_CARD": abricate_CARD,
    "abricate_NCBI": abricate_NCBI,
    "abricate_RESFINDER": abricate_RESFINDER,
    "amrfinderplus_NCBI": amrfinderplus_NCBI,
    "rgi_CARD": rgi_CARD
}

In [4]:
#filter to see if the found gene is a BLEE or CPO.
for key, value in all_reports.items():
   all_reports[key]['gene_type'] = all_reports[key]['gene_symbol_modified'].apply(determine_gene_type)
   
#filter for BLEEs and CPOs
all_reports_blees_cpos = {}
for key in all_reports.keys():
    all_reports_blees_cpos[key] = all_reports[key][(all_reports[key]['gene_type'] == "BLEE") | (all_reports[key]['gene_type'] == "CPO")]

In [5]:
# Initialize an empty set to store unique gene_symbol_modified values
unique_genes = set()

# Iterate through each DataFrame in the all_reports dictionary
for key, df in all_reports_blees_cpos.items():
    # Filter the DataFrame for rows where sequence_identity is 99%
    filtered_df = df[df['sequence_identity'] >= 99]
    # Add the unique gene_symbol_modified values to the set
    unique_genes.update(filtered_df['gene_symbol_modified'].unique())

# Convert the set to a list if needed
unique_genes_list = list(unique_genes)

# Print the unique gene_symbol_modified values
print(unique_genes_list)

['CTX_M_163', 'TEM_122', 'OXA_247', 'SHV_155', 'CMY_2', 'CMY_59', 'SHV_145', 'SHV_105', 'SHV_51', 'SHV_28', 'SHV_38', 'SHV_1', 'SHV_67', 'SHV_120', 'SHV_106', 'TEM_1', 'SHV_134', 'TEM_90', 'OXA_836', 'SHV_172', 'SHV_98', 'TEM_30', 'TEM_47', 'SHV_11', 'OXA_405', 'SHV_161', 'SHV_33', 'CTX_M_139', 'SHV_36', 'SHV_70', 'CTX_M_187', 'OXA_17', 'NDM_9', 'TEM_57', 'TEM_206', 'VIM_1', 'TEM_185', 'OXA_48', 'TEM_79', 'CTX_M_15', 'SHV_158', 'VIM_86', 'SHV_12', 'NDM_23', 'SHV_187', 'TEM_150', 'TEM_95', 'KPC_3', 'SHV_182', 'SHV_8']


In [ ]:
# Initialize an empty DataFrame with unique genes as rows
result_df = pd.DataFrame(unique_genes_list, columns=['gene_symbol_modified'])

# Iterate through each DataFrame in the all_reports dictionary again to populate the result DataFrame
for key, df in all_reports.items():
    # Filter the DataFrame for rows where gene_symbol_modified is in the unique_genes_list
    filtered_df = df[df['gene_symbol_modified'].isin(unique_genes_list)]
    # Group by gene_symbol_modified and get the maximum sequence_identity for each gene
    mean_seq_identity = filtered_df.groupby('gene_symbol_modified')['sequence_identity'].mean()
    # Convert the Series to a DataFrame
    mean_seq_identity_df = mean_seq_identity.reset_index()
    # Rename the sequence_identity column to the tool name
    mean_seq_identity_df = mean_seq_identity_df.rename(columns={'sequence_identity': key})
    # Merge the mean_seq_identity_df with the result DataFrame
    result_df = result_df.merge(mean_seq_identity_df, how='left', on='gene_symbol_modified')

for row in result_df.iterrows():
    result_df['gene_type'] = result_df['gene_symbol_modified'].apply(determine_gene_type)


# Print the result DataFrame
result_df.head()

# Save the result DataFrame to a TSV file
result_df.to_csv('output_dir/20240808_result_df_mean_seqID_tool.tsv', sep='\t', index=True)

# ANEX:
1. Descriptive parameters of the reports: sequence ID (for all reports) and coverage depth (only for ARIBA)

In [ ]:
#saving descriptive parameters of the reports.
for key, value in all_reports.items():
    print(key)
    print(value['sequence_identity'].describe())

for key, value in reports_ariba.items():
    print(key)
    print(value['coverage_depth'].describe())

In [ ]:
#make a list of ALL found genes among all softwares and databases. This will be the Y axis in out heatmap plot.
all_genes = []
for value in all_reports.values():
    for gene in value["gene_symbol_modified"]:
        if gene not in all_genes:
            all_genes.append(gene)
all_genes
print(len(all_genes))


with open('all_genes.txt', 'w')as f:
    for gene in all_genes:
        f.write(f"{gene}\n")

In [ ]:
#make a list of the f99 genes among all softwares and databases. This will be the Y axis in out heatmap plot.
all_genes = []
for value in all_reports_filtered99.values():
    for gene in value["gene_symbol_modified"]:
        if gene not in all_genes:
            all_genes.append(gene)
all_genes
print(len(all_genes))


with open('all_genes_filtered99.txt', 'w')as f:
    for gene in all_genes:
        f.write(f"{gene}\n")